In [ ]:
import sys
sys.path.append('..')
from core.utilities.notebook_auto_setup import *
notebook_set_up(LogLvl.SPARSE)

# Initialization

In [ ]:
data_manager, set_data = load_set_data()
card_dict = set_data.SET_METADATA.CARD_DICT

## Tier List Analysis

### Functions

In [ ]:
tier_to_rank = {
    "A+": 12,
    "A": 11,
    "A-": 10,
    "B+": 9,
    "B": 8,
    "B-": 7,
    "C+": 6,
    "C": 5,
    "C-": 4,
    "D+": 3,
    "D": 2,
    "D-": 1,
    "F": 0,
    "SB": None,
    "TBD": None
}


rank_to_tier = {v: k for k, v in tier_to_rank.items()}

In [ ]:
def get_stats_grades(deck_color: str = ''):
    frame = set_data.BO1.card_frame(deck_color=deck_color, summary=True).copy()
    mu, std = norm.fit(frame['GIH WR'])
    frame['STD'] = norm.cdf(frame['GIH WR'], mu, std) * 100
    frame = frame['GIH WR']
    
    range_map = [
        frame['STD'].between(99, 100),
        frame['STD'].between(95, 99),
        frame['STD'].between(90, 95),
        frame['STD'].between(85, 90),
        frame['STD'].between(76, 85),
        frame['STD'].between(68, 76),
        frame['STD'].between(57, 68),
        frame['STD'].between(45, 57),
        frame['STD'].between(36, 45),
        frame['STD'].between(27, 36),
        frame['STD'].between(17, 27),
        frame['STD'].between(5, 17),
        frame['STD'].between(0, 5)
    ]
    
    frame['Tier'] = np.select(range_map, ranks, 0)
    frame['Rank'] = frame['Tier'].map(tier_to_rank).astype('Int64')
    
    return frame

In [ ]:
def frame_from_url(url, name):
    def gen_card_dict(data):
        return {
            'Card': data['name'],
            'Tier': data['tier'],
            'Rank': tier_to_rank[data['tier']],
            'Synergy': data['flags']['synergy'],
            'Buildaround': data['flags']['buildaround']
        }

    fetcher = Request17Lands()
    raw_data = fetcher.get_tier_list(url.replace("https://www.17lands.com/tier_list/", ""))
    tier_data = {card_data['name']: gen_card_dict(card_data) for card_data in raw_data}
    frame = pd.DataFrame.from_dict(tier_data, orient="index")
    frame.index.name = name
    return frame

In [ ]:
def merge_rankings(frame_list, card_dict):
    # Create a frame with only card names.
    frame = pd.DataFrame()
    frame.index.name = 'Card'
    
    # Get each user's coverted ranks as ints. 
    ranks = pd.DataFrame()
    for indiv in frame_list:
        ranks[indiv.index.name] = indiv['Rank'].astype('Int64')
        frame[indiv.index.name] = indiv['Rank'].astype('Int64')
        
    # Calculate the general stats and append them.
    frame['mean'] = ranks.mean(axis=1)
    frame['max'] = ranks.max(axis=1)
    frame['min'] = ranks.min(axis=1)
    frame['range'] = frame['max'] - frame['min']
    frame['std'] = ranks.std(axis=1).round(1)
    
    # Get the difference of squares distance to figure out most 'controversial' cards.
    dist = pd.DataFrame()
    for indiv in frame_list:
        dist[indiv.index.name] = abs(frame['mean'] - ranks[indiv.index.name])
    frame['dist'] = dist.mean(axis=1).round(1)

    
    series = frame.index.to_series()
    frame['Image'] = series.map({card.NAME: card.NAME for card in card_dict.values()})
    frame['Cast Color'] = series.map({card.NAME: card.CAST_IDENTITY for card in card_dict.values()})
    frame['Color'] = series.map({card.NAME: card.COLOR_IDENTITY for card in card_dict.values()})
    frame['Rarity'] = series.map({card.NAME: card.RARITY for card in card_dict.values()})
    frame['CMC'] = series.map({card.NAME: card.CMC for card in card_dict.values()})
    
    cols = list(frame.columns)
    frame = frame[['Image', 'CMC', 'Rarity', 'Color', 'Cast Color'] + cols[:-5]]

    return frame

In [ ]:
def style_frame(sub_frame):
    def hoverable(card_name):
        try:
            card = card_dict[card_name]
            hmtl = '<style>.hover_img a { position:relative; }\n' + \
            '.hover_img a span { position:absolute; display:none; z-index:300; }\n' + \
            '.hover_img a:hover span { display:block; height: 300px; width: 300px; overflow: visible; margin-left: -175px; }</style>\n' + \
            f'<div class="hover_img">\n' + \
            f'<a href="#">{card_name}<span><img src="{card.IMAGE_URL}" alt="image"/></span></a>\n' + \
            f'</div>'
            return hmtl
        except KeyError:
            return f"ERROR - {card_name}"
     
    def format_short_float(val):
        return '{:.1f}'.format(val)
    
    def format_long_float(val):
        return '{:.3f}'.format(val)
    
    sub_frame = sub_frame.style.format({
        'Image': hoverable,
        'mean': format_long_float,
        'max': format_short_float,
        'min': format_short_float,
        'range': format_short_float,
        'dist': format_short_float,
        'std': format_short_float,
    })
    
    return sub_frame


def display_frame(frame, order='mean', filters=None):
    pd.set_option('display.max_rows', 300)
    
    if filters is not None:
        filt_frame = pd.DataFrame()
        for f in filters:
            filt_frame[id(f)] = f(frame)
        frame = frame[filt_frame.T.all()]
            
    frame = frame.sort_values(order, ascending=False)
            
    return style_frame(frame)

In [ ]:
def get_avgs(frame):
    colors = 'WUBRG'
    rarities = 'MRUC'
    avgs = dict()
    
    for r in rarities:
        color_frame = pd.DataFrame()
        
        for c in colors:
            filt_frame = pd.DataFrame()
            filt_frame['color'] = cast_color_filter(c)(frame)
            filt_frame['rarity'] = rarity_filter(r)(frame)
            working = frame[filt_frame.T.all()].copy(True)
            working = working.drop(['Image', 'CMC', 'Rarity', 'Color', 'Cast Color', 'max', 'min', 'range', 'dist'], axis=1)
            working = working.dropna(how='all', axis=1)
            color_frame[c] = working.mean().round(1)
            
        avgs[r] = color_frame.T
    
    ret = pd.concat(avgs)
    names = list(ret.columns)[:-1]
    
    ret['max'] = ret.max(axis=1)
    ret['min'] = ret.min(axis=1)
    ret['range'] = ret['max'] - ret['min']
    
    # Get the difference of squares distance to figure out most 'controversial' cards.
    dist = pd.DataFrame()
    for name in names:
        dist[name] = abs(ret['mean'] - ret[name])
    dist['dist'] = dist.mean(axis=1)
    ret['dist'] = dist['dist'].round(2)

    return ret

In [ ]:
def split_by_rarity(frame, ordering='mean'):
    rarities = "MRUC"
    by_rarities = [display_frame(frame, order=ordering, filters=[rarity_filter(r)]).data for r in rarities]
    return style_frame(pd.concat(by_rarities))

In [ ]:
raise Exception("Stopping Auto-Execute")

### Data Analysis

After loading all of the data, we crunch it to figure out the average grade that each contributer has given to cards. While not totally necessary, this helps anchor evaluations between contrinutors, as one person's C may be another person's C+. 

In [ ]:
dmu_pairs = [
    ("https://www.17lands.com/tier_list/03ab10d25d8841f8aef0aa90b30c434b", "Zac"),
    ("https://www.17lands.com/tier_list/e12ee0b1fadc4ab7b8de4c3730878a90", "Chord"),
    ("https://www.17lands.com/tier_list/3078f70b9a0d415ebf9f555439f5aedb", "Klarm"),
    ("https://www.17lands.com/tier_list/a1ae9a695abb4921866f7f4a9a22e21f", "Phyrre"),
    ("https://www.17lands.com/tier_list/19ee3198a63b499bb7c25e6ceaae00ba", "Nomad"),
    ("https://www.17lands.com/tier_list/f1b015031fd741268c5228d4d8435938", "Catharsis"),
    ("https://www.17lands.com/tier_list/e854e4a17f5147ad9908c5f6e5d2e87c", "Rewind"),
    ("https://www.17lands.com/tier_list/1d343c3e1f0d4cfd8c356cdaa42bb329", "Davis"),
]

bro_pairs = [
    #("https://www.17lands.com/tier_list/bc43f79ac9d34b11bc43d904a97b8795", "Zac"),
    ("https://www.17lands.com/tier_list/b8d4ba9d1bad49828bfa6371f6b4f09b", "Chord"),
    ("https://www.17lands.com/tier_list/61fdaf8a13164ec0a87c954f0ef959e5", "Ncaa"),
]


one_pairs = [
    ("https://www.17lands.com/tier_list/45a3a3a84d9f46178d6750ff96d85f8c", "Zac"),
    ("https://www.17lands.com/tier_list/1f286922c200438493eca0c7c2cd52de", "Klarm"),
    ("https://www.17lands.com/tier_list/37da6967cc464c59aabb789061cf54bd", "CardboardNomad"),
    ("https://www.17lands.com/tier_list/6a4b4990e9324d018509bbdf8611c84c", "Glassblowings"),
    ("https://www.17lands.com/tier_list/8df037923b984400897eacb998646a2d", "Arcyl"),
    ("https://www.17lands.com/tier_list/40c76666327a4d969bc139d32beb48ea", "CryoGyro"),
    ("https://www.17lands.com/tier_list/5bd47cffc044433dba04bf705d60739d", "Prosperity"),
    ("https://www.17lands.com/tier_list/b9ac0f6bbd86442e98f861d29d52d689", "Ncaa"),
    ("https://www.17lands.com/tier_list/48367d8421e24aa0adaa1b9ee23884e8", "ztm"),
]

pairs = one_pairs
indiv_frames = [frame_from_url(*pair) for pair in pairs]

In [ ]:
frame = merge_rankings(indiv_frames, card_dict)
frame

In [ ]:
# base_frame.to_csv("C:\\Users\\Zachary\\Downloads\\TierlistSummary.csv", encoding='utf-8')
frame.mean()

### 'Controversial' Cards

Here, we separate the cards by rarity, and order them by the sum of the difference between each rating and the mean. This is saved in the `dist` column, and the higher the `dist` the lesss people agree on the rating of a card. If the `dist` is zero, everyone has given the card the same grade. The value of dist can either mean that some people disagree and most agree, or everyone disagrees to varying degrees.

To me, anyting with a dist less than or equal to 0.6 is pretty agreed upon, and anything higher than 1.0 is fairly contentious.

In [ ]:
split_by_rarity(frame, 'dist')

### Top Cards

This is the short-list of top commons and uncommons, ordered by colour, based on average rating of contributors.

In [ ]:
ordering = 'mean'
top = list()

for c in 'WUBRG':
    top.append(display_frame(frame, order=ordering, filters=[rarity_filter('U'), cast_color_filter(c)]).data.head(3))
    top.append(display_frame(frame, order=ordering, filters=[rarity_filter('C'), cast_color_filter(c)]).data.head(5))

display = pd.concat(top)
style_frame(display)

This is the entire list of cards, ordered by rarity, then by average rating amongst contributers.

In [ ]:
split_by_rarity(frame, 'mean')

### Color Ratings

This is a little rougher, and more difficult to parse, but this is the avreage grade for each contirbutor given to cards of 'x' colour and 'y' rarity. it should serve as a rough estimation of how strong each person thinks a given colour is, though doesn't necessarily translate well to colour combinations.

In [ ]:
get_avgs(frame)

### Gut-Check

The two selections below are mainly for my benefit (though I can send any contributor a personalized copy if they ask!), to check my opions of cards against what everyone else thinks. Any card I have graded more than a grade higher or lower than average is shown here, to give a more concrete idea of where _I_ disagree, than where _people_ disagree.

In [ ]:
play_frame = frame.copy(True)
play_frame['diff'] = frame['Zac'] - frame['mean']
style_frame(play_frame[play_frame['diff'] > 1].sort_values('diff', ascending=False))

In [ ]:
play_frame = frame.copy(True)
play_frame['diff'] = frame['Zac'] - frame['mean']
style_frame(play_frame[play_frame['diff'] < -1].sort_values('diff', ascending=True))

## BRO Specific Code

## DMU Specific Code

These are the rankings of all the non-basic lands.
In short:
- Gx taplands are highest rated
- UR tapland is slightly ahead of other non-GX tap lands
- Taplands are ahead of painlands

In [ ]:
lands = [
     'Adarkar Wastes',
     'Caves of Koilos',
     'Shivan Reef',
     'Yavimaya Coast',
     'Sulfurous Springs',
     'Karplusan Forest',
     'Plaza of Heroes',
     'Thran Portal',
     'Crystal Grotto',
     'Idyllic Beachfront',
     'Sunlit Marsh',
     'Sacred Peaks',
     'Radiant Grove',
     'Contaminated Aquifer',
     'Molten Tributary',
     'Tangled Islet',
     'Geothermal Bog',
     'Haunted Mire',
     'Wooded Ridgeline',
]
style_frame(frame.loc[lands])

## ONE Specific Code

In [ ]:
cards = [
    '',
]
style_frame(frame.loc[lands])

### Specific Checks

These are a list of specific cards I wanted to compare, because they either seem interesting, are cards that feel similar-but-different. Feel free to ask me to add certain cards to this list if you're curious

In [ ]:
frame.loc[['Phyrexian Obliterator']]